In [1]:
from emg_fatigue.utils.load_emg_data import load_all_participant_data
from emg_fatigue.utils.process_emg_data import process_all_participant_data
from emg_fatigue.utils.create_loocv_dataset import create_loocv_dataset
from emg_fatigue.modeling.train import train_model
from emg_fatigue.modeling.evaluate import evaluate_model
from emg_fatigue.modeling.build.rnn_model import build_lstm_model
from emg_fatigue.modeling.build.mlp_model import build_mlp_model
from emg_fatigue.modeling.build.transformer_model import build_transformer_model
from emg_fatigue.modeling.build.gru_model import build_gru_model
from emg_fatigue.plots.visualize_model_predictions import visualize_model_predictions
from emg_fatigue.config import BATCH_SIZE, PADDING_VALUE

2025-04-13 13:34:42.632953: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-13 13:34:42.633476: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-13 13:34:42.636651: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-13 13:34:42.644297: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744544082.657820  830110 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744544082.66

In [2]:
raw_data = load_all_participant_data()
processed_data = process_all_participant_data(participant_data=raw_data)


# --- Create LOO-CV Datasets ---
all_participant_ids = list(processed_data.keys())
train_ids = all_participant_ids[:-3] 
val_ids = all_participant_ids[-3:-1]
test_ids = [all_participant_ids[-1]]

train_ds, val_ds, test_ds, input_shape, output_shape, norm_mean, norm_std = create_loocv_dataset(
    processed_data=processed_data,
    train_participant_ids=train_ids,
    validation_participant_ids=val_ids,
    test_participant_ids=test_ids,
    batch_size=BATCH_SIZE,
    padding_value=PADDING_VALUE,
    normalize=True,
    augment=True,
    time_param=10,
    time_masks=1,
    freq_param=10,
    freq_masks=1,
    augmentation_factor=1
)



2025-04-13 13:34:50.290 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:162 - Processing data for datasets...
2025-04-13 13:34:50.290 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:170 -   Processing train set (9 participants)...
2025-04-13 13:34:50.291 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:170 -   Processing val set (2 participants)...
2025-04-13 13:34:50.291 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:170 -   Processing test set (1 participants)...
2025-04-13 13:34:50.291 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:233 - 
Found 513 frequency bins (features).
2025-04-13 13:34:50.291 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:234 - Maximum sequence length across all sets: 166.
2025-04-13 13:34:50.291 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:238 - Calculating normalization statistics 

2025-04-13 13:34:50.387136: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


2025-04-13 13:34:51.012 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:294 -   Data augmentation complete. Training samples: 144 → 288
2025-04-13 13:34:51.012 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:299 - Padding sequences...
2025-04-13 13:34:51.042 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:333 -   Padded Train data shape: X=(288, 166, 513), y=(288, 166)
2025-04-13 13:34:51.047 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:333 -   Padded Val data shape: X=(32, 166, 513), y=(32, 166)
2025-04-13 13:34:51.049 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:333 -   Padded Test data shape: X=(16, 166, 513), y=(16, 166)
2025-04-13 13:34:51.049 | INFO     | emg_fatigue.utils.create_loocv_dataset:create_loocv_dataset:347 -   Standardized Training data shape: X=(288, 166, 513), y=(288, 166)
2025-04-13 13:34:51.049 | INFO     | emg_fatigue.utils.create_loo

In [3]:
# model, model_name = build_lstm_model(input_shape=input_shape, test_id=test_ids, padding_value=PADDING_VALUE)

# model, model_name = build_mlp_model(input_shape=input_shape, test_id=test_ids, padding_value=PADDING_VALUE)

# model, model_name = build_transformer_model(input_shape=input_shape, test_id=test_ids, padding_value=PADDING_VALUE)

model, model_name = build_gru_model(input_shape=input_shape, test_id=test_ids, padding_value=PADDING_VALUE)


train_model(model=model, train_ds=train_ds, val_ds=val_ds, model_name=model_name, epochs=200)
evaluate_model(model=model, test_ds=test_ds, model_name=model_name)

2025-04-13 13:34:51.140 | INFO     | emg_fatigue.modeling.build.transformer_model:build_transformer_model:234 - Building Transformer model for test participant: P012
2025-04-13 13:34:51.496 | INFO     | emg_fatigue.modeling.train:train_model:48 - Model will be saved to: /home/lg519/Projects/Muscle_Fatigue_Research/emg_fatigue/models/transformer_P012.keras
2025-04-13 13:34:51.496 | INFO     | emg_fatigue.modeling.train:train_model:77 - Starting model training for up to 200 epochs...
Epoch 1/200


/home/lg519/.cache/pypoetry/virtualenvs/emg-fatigue-YbMeuwX9-py3.12/lib/python3.12/site-packages/keras/src/layers/layer.py:940: UserWarning: Layer 'squeeze_output' (of type Lambda) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - loss: 2035.2405 - mae: 32.3365
Epoch 1: val_loss improved from inf to 1849.78296, saving model to /home/lg519/Projects/Muscle_Fatigue_Research/emg_fatigue/models/transformer_P012.keras
9/9 ━━━━━━━━━━━━━━━━━━━━ 13s 358ms/step - loss: 2028.6211 - mae: 32.3272 - val_loss: 1849.7830 - val_mae: 32.6276 - learning_rate: 0.0010
Epoch 2/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 1723.4550 - mae: 30.9706
Epoch 2: val_loss improved from 1849.78296 to 1680.10254, saving model to /home/lg519/Projects/Muscle_Fatigue_Research/emg_fatigue/models/transformer_P012.keras
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step - loss: 1721.3630 - mae: 30.9782 - val_loss: 1680.1025 - val_mae: 31.8045 - learning_rate: 0.0010
Epoch 3/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 1562.8542 - mae: 30.2554
Epoch 3: val_loss improved from 1680.10254 to 1550.37903, saving model to /home/lg519/Projects/Muscle_Fatigue_Research/emg_fatigue/models/transformer_P012.keras
9/9 ━━━━━━━━

{'loss': 220.07284545898438, 'mae': 9.845563888549805}

In [4]:
visualize_model_predictions(
    model=model,
    model_name=model_name,
    processed_data=processed_data,
    test_participant_ids=test_ids,
    input_shape=input_shape,
    norm_mean=norm_mean,
    norm_std=norm_std
)

2025-04-13 13:35:52.322 | INFO     | emg_fatigue.plots.visualize_model_predictions:visualize_model_predictions:59 - Starting prediction visualization for 1 test participants using model 'transformer_P012'.
2025-04-13 13:35:52.326 | INFO     | emg_fatigue.plots.visualize_model_predictions:visualize_model_predictions:63 - Normalization statistics provided. Applying normalization to test data.
2025-04-13 13:35:52.326 | INFO     | emg_fatigue.plots.visualize_model_predictions:visualize_model_predictions:77 - Visualizing predictions for participant P012...
2025-04-13 13:35:52.326 | INFO     | emg_fatigue.plots.visualize_model_predictions:visualize_model_predictions:100 -   Processing recording 1/8 for P012 left...
2025-04-13 13:35:52.327 | DEBUG    | emg_fatigue.plots.visualize_model_predictions:visualize_model_predictions:140 -   Applied normalization to spectrogram for P012 left Rec 1
2025-04-13 13:35:56.338 | INFO     | emg_fatigue.plots.visualize_model_predictions:visualize_model_predic